In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from metrics.Multilabel_classification_metrics import Precision, Accuracy, Recall, F1Measure, Hamming_Loss
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import ClassifierChain
import pickle
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score

### Load Data

In [3]:
# Dataframe
path_df = "../Pickles/multilabel_binarizer.pickle"
with open(path_df, 'rb') as data:
    multilabel_binarizer = pickle.load(data)

# features_train
path_features_train = "../Pickles/features_train.pickle"
with open(path_features_train, 'rb') as data:
    features_train = pickle.load(data)

# labels_train
path_labels_train = "../Pickles/labels_train.pickle"
with open(path_labels_train, 'rb') as data:
    labels_train = pickle.load(data)

# features_test
path_features_test = "../Pickles/features_test.pickle"
with open(path_features_test, 'rb') as data:
    features_test = pickle.load(data)

# labels_test
path_labels_test = "../Pickles/labels_test.pickle"
with open(path_labels_test, 'rb') as data:
    labels_test = pickle.load(data)

### Train model

In [8]:
clf = ClassifierChain(MultinomialNB())
clf.fit(features_train,labels_train)
y_predictions = clf.predict(features_test)

In [9]:
multilabel_binarizer.inverse_transform(y_predictions)[95]

('Non-fiction',)

In [10]:
F1Measure(labels_test, y_predictions)

0.21389771858053763

In [12]:
y_pred_prob = clf.predict_proba(features_test)
y_pred_prob

array([[1.72147443e-02, 1.51058648e-02, 7.16233608e-03, ...,
        2.08974561e-02, 2.67024022e-02, 9.92767704e-02],
       [1.76718943e-02, 4.16615476e-02, 5.92377236e-03, ...,
        3.17072751e-02, 5.53636296e-02, 5.21550105e-02],
       [1.37176395e-02, 3.43254111e-03, 4.62319807e-03, ...,
        3.13019134e-02, 9.40308113e-02, 4.12606396e-01],
       ...,
       [2.72033909e-02, 3.98331641e-03, 4.96796495e-03, ...,
        9.57766929e-03, 1.33925867e-02, 5.38736502e-01],
       [3.74163417e-02, 1.70208273e-02, 5.13934155e-03, ...,
        1.97324420e-02, 2.56839853e-02, 3.44652738e-02],
       [3.28612184e-02, 1.37359708e-02, 3.20471959e-03, ...,
        1.02788247e-04, 7.47673023e-04, 9.30191511e-02]])

In [13]:
t = 0.2485 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

In [14]:
F1Measure(labels_test, y_pred_new)

0.4806852614366982

In [15]:
accuracy_score(labels_test, y_pred_new)

0.051728832017424446

In [16]:
Accuracy(labels_test, y_pred_new)

0.3664052135056782

In [17]:
Precision(labels_test, y_pred_new)

0.6395589436427986

In [18]:
Recall(labels_test, y_pred_new)

0.4374638611229952